In [1]:
from pathlib import Path
import os
import sys
import gc
import shutil
import json
import math
import uuid
from collections import defaultdict
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, NamedTuple, Callable, Iterable, Set, Optional, Any
import textstat
import scml
from scml import pandasx as pdx
import lalaes2 as mylib

In [2]:
version = "01"
n_splits = 5
text_col = "clean_text"
features = []

In [3]:
class ModelConf(NamedTuple):
    name: str
    directory: Path
    model_max_length: int
    batch_size: int
    model_class: str = "auto"


models_conf: List[ModelConf] = [
    ModelConf(
        name="deberta_base", 
        directory=Path("models/aes2/deberta_v3_base/20240610_172052"),
        model_max_length=512,
        batch_size=128,
    ),
]

In [4]:
tim = scml.Timer()
tim.start()
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()

In [5]:
df = pd.read_parquet(f"input/val_{version}.parquet")
df = df.drop(columns=["source", "str_level"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 866 entries, 0 to 865
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   essay_id    866 non-null    object
 1   score       866 non-null    int8  
 2   clean_text  866 non-null    object
dtypes: int8(1), object(2)
memory usage: 14.5+ KB


# Base Model Inference

In [6]:
for mc in models_conf:
    print(mc.name)
    df[mc.name] = mylib.predict_holistic_score(
        ds=mylib.Aes2Dataset(
            tokenizer=AutoTokenizer.from_pretrained(mc.directory, model_max_length=mc.model_max_length),
            texts=df[text_col].tolist(),
        ),
        model=AutoModelForSequenceClassification.from_pretrained(mc.directory),
        batch_size=mc.batch_size,
        device=torch.device("cuda:1"),
        progress_bar=True,
    )
    features.append(mc.name)

deberta_base


/home/pankun/.local/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
predict hms score: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:10<00:00,  1.54s/it]


# Character & Word-level Features

In [7]:
%%time
col = "cw_len"
df[col] = df[text_col].str.len()
features.append(col)

CPU times: user 977 µs, sys: 247 µs, total: 1.22 ms
Wall time: 1.16 ms


In [8]:
def digit_frac(row) -> float:
    return mylib.digit_frac(row[text_col])


def letter_frac(row) -> float:
    return mylib.letter_frac(row[text_col])


def space_frac(row) -> float:
    return mylib.space_frac(row[text_col])


def punc_frac(row) -> float:
    return mylib.punc_frac(row[text_col])


def upper_frac(row) -> float:
    return mylib.upper_frac(row[text_col])


def repeat_char_frac(row) -> float:
    return mylib.repeat_char_frac(row[text_col])


def repeat_substring_frac(row) -> float:
    return mylib.repeat_substring_frac(row[text_col])


def unique_word_frac(row) -> float:
    return mylib.unique_word_frac(row[text_col])


sf = mylib.StopwordFraction()


def stopword_frac(row) -> float:
    return sf(row[text_col])



cw_fns: List[Tuple[str, Callable]] = [
    ("cw_digit_frac", digit_frac),
    ("cw_letter_frac", letter_frac),
    ("cw_space_frac", space_frac),
    ("cw_punc_frac", punc_frac),
    ("cw_upper_frac", upper_frac),
    ("cw_repeat_char_frac", repeat_char_frac),
    ("cw_repeat_substring_frac", repeat_substring_frac),
    ("cw_unique_word_frac", unique_word_frac),
    ("cw_stopword_frac", stopword_frac),
]   
for col, fn in cw_fns:
    print(col)
    df[col] = df.progress_apply(fn, axis=1)
    features.append(col)
df[features] = df[features].astype(np.float32)

cw_digit_frac


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 866/866 [00:00<00:00, 15239.02it/s]


cw_letter_frac


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 866/866 [00:00<00:00, 13538.54it/s]


cw_space_frac


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 866/866 [00:37<00:00, 23.36it/s]


cw_unique_word_frac


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 866/866 [00:00<00:00, 33378.06it/s]


cw_stopword_frac


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 866/866 [00:00<00:00, 28849.04it/s]


# Textstat Features

In [9]:
def syllable_count(row) -> int:
    return textstat.syllable_count(row[text_col])


def lexicon_count(row) -> int:
    return textstat.lexicon_count(row[text_col])


def sentence_count(row) -> int:
    return textstat.sentence_count(row[text_col])


def syllables_per_word(row) -> float:
    return row["ts_syllable_count"] / (row["ts_lexicon_count"] + 1)


def syllables_per_sent(row) -> float:
    return row["ts_syllable_count"] / (row["ts_sentence_count"] + 1)


def words_per_sent(row) -> float:
    return row["ts_lexicon_count"] / (row["ts_sentence_count"] + 1)


def flesch_reading_ease(row) -> float:
    return textstat.flesch_reading_ease(row[text_col])


def flesch_kincaid_grade(row) -> float:
    return textstat.flesch_kincaid_grade(row[text_col])


def gunning_fog(row) -> float:
    return textstat.gunning_fog(row[text_col])


def smog_index(row) -> float:
    return textstat.smog_index(row[text_col])


def automated_readability_index(row) -> float:
    return textstat.automated_readability_index(row[text_col])


def coleman_liau_index(row) -> float:
    return textstat.coleman_liau_index(row[text_col])


def linsear_write_formula(row) -> float:
    return textstat.linsear_write_formula(row[text_col])


def dale_chall_readability_score(row) -> float:
    return textstat.dale_chall_readability_score(row[text_col])


def text_standard(row) -> float:
    return textstat.text_standard(row[text_col], float_output=True)


def mcalpine_eflaw(row) -> float:
    return textstat.mcalpine_eflaw(row[text_col])


textstat_fns: List[Tuple[str, Callable]] = [
    ("ts_syllable_count", syllable_count),
    ("ts_lexicon_count", lexicon_count),
    ("ts_sentence_count", sentence_count),
    ("ts_syllables_per_word", syllables_per_word),
    ("ts_syllables_per_sent", syllables_per_sent),
    ("ts_words_per_sent", words_per_sent),
    ("ts_flesch_reading_ease", flesch_reading_ease),
    ("ts_flesch_kincaid_grade", flesch_kincaid_grade),
    ("ts_gunning_fog", gunning_fog),
    ("ts_smog_index", smog_index),
    ("ts_automated_readability_index", automated_readability_index),
    ("ts_coleman_liau_index", coleman_liau_index),
    ("ts_linsear_write_formula", linsear_write_formula),
    ("ts_dale_chall_readability_score", dale_chall_readability_score),
    ("ts_text_standard", text_standard),
    ("ts_mcalpine_eflaw", mcalpine_eflaw),
]    
for col, fn in textstat_fns:
    print(col)
    df[col] = df.progress_apply(fn, axis=1)
    features.append(col)
df[features] = df[features].astype(np.float32)

ts_syllable_count


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 866/866 [00:00<00:00, 2472.00it/s]


ts_lexicon_count


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 866/866 [00:00<00:00, 27821.31it/s]


ts_sentence_count


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 866/866 [00:00<00:00, 2894.90it/s]


ts_text_standard


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 866/866 [00:01<00:00, 826.18it/s]


ts_mcalpine_eflaw


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 866/866 [00:00<00:00, 9131.46it/s]


# Train/Test Split

In [10]:
splitter = StratifiedKFold(n_splits=n_splits, shuffle=True)
dummy = np.zeros(len(df))
col = "is_test"
df[col] = 0
for _, vi in splitter.split(dummy, y=df["score"].tolist()):
    df[col].iloc[vi] = 1
    break
df[col] = df[col].astype(np.int8)

/tmp/ipykernel_76796/2611437346.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df[col].iloc[vi] = 1
/tmp/ipykernel_76796/2611437346.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See th

# Review Data

In [11]:
df[features] = df[features].astype(np.float32)
df = df.drop(columns=["clean_text"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 866 entries, 0 to 865
Data columns (total 30 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   essay_id                         866 non-null    object 
 1   score                            866 non-null    int8   
 2   deberta_base                     866 non-null    float32
 3   cw_len                           866 non-null    float32
 4   cw_digit_frac                    866 non-null    float32
 5   cw_letter_frac                   866 non-null    float32
 6   cw_space_frac                    866 non-null    float32
 7   cw_punc_frac                     866 non-null    float32
 8   cw_upper_frac                    866 non-null    float32
 9   cw_repeat_char_frac              866 non-null    float32
 10  cw_repeat_substring_frac         866 non-null    float32
 11  cw_unique_word_frac              866 non-null    float32
 12  cw_stopword_frac      

In [12]:
df.describe(percentiles=percentiles)

,score,deberta_base,cw_len,cw_digit_frac,cw_letter_frac,cw_space_frac,cw_punc_frac,cw_upper_frac,cw_repeat_char_frac,cw_repeat_substring_frac,cw_unique_word_frac,cw_stopword_frac,ts_syllable_count,ts_lexicon_count,ts_sentence_count,ts_syllables_per_word,ts_syllables_per_sent,ts_words_per_sent,ts_flesch_reading_ease,ts_flesch_kincaid_grade,ts_gunning_fog,ts_smog_index,ts_automated_readability_index,ts_coleman_liau_index,ts_linsear_write_formula,ts_dale_chall_readability_score,ts_text_standard,ts_mcalpine_eflaw,is_test
count,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.00000,866.000000,866.000000
mean,2.948037,3.145202,2060.635010,0.002611,0.795851,0.181177,0.020361,0.018148,0.015119,0.002907,0.533583,0.504496,509.431885,370.844116,19.625866,1.362833,26.972654,19.788095,68.661377,9.188222,10.897321,10.107390,11.009238,8.253776,10.499711,7.593972,9.80485,31.437298,0.200924
std,1.046139,0.929500,849.165894,0.003620,0.011515,0.010704,0.006710,0.009073,0.004385,0.007891,0.075234,0.048059,212.012878,147.202759,8.809137,0.092970,20.295984,15.702071,32.334080,12.169928,12.454148,1.943423,15.534221,1.855127,5.296445,1.734166,12.05920,45.399860,0.400923
min,1.000000,1.406469,732.000000,0.000000,0.743163,0.155241,0.000000,0.000000,0.004212,0.000000,0.264228,0.354054,190.000000,151.000000,1.000000,1.108434,10.962963,8.500000,-628.880005,2.200000,3.910000,0.000000,2.800000,3.190000,2.800000,5.380000,0.00000,12.100000,0.000000
1%,1.000000,1.488527,843.000000,0.000000,0.765810,0.159538,0.006005,0.000906,0.006511,0.000000,0.361339,0.385154,205.600000,159.300000,4.000000,1.153609,13.469216,10.195938,37.691500,3.265000,5.416500,5.765000,3.565000,3.959500,4.235606,5.800000,5.00000,14.765000,0.000000
5%,1.000000,1.790974,969.000000,0.000000,0.776374,0.164993,0.010542,0.006928,0.008457,0.000000,0.410804,0.427421,238.500000,180.000000,8.000000,1.208756,16.406060,12.396826,53.077499,4.800000,6.655000,7.400000,5.400000,5.155000,5.458333,6.172500,6.00000,18.425000,0.000000
10%,2.000000,1.988272,1087.500000,0.000000,0.781379,0.168216,0.012364,0.009094,0.009790,0.000000,0.439701,0.446998,269.500000,202.000000,9.500000,1.241224,17.884259,13.362500,57.400002,5.400000,7.280000,7.900000,6.150000,5.840000,6.000000,6.400000,6.00000,19.900000,0.000000
20%,2.000000,2.232888,1314.000000,0.000000,0.786875,0.171930,0.015158,0.011138,0.011118,0.000000,0.466775,0.467442,322.000000,241.000000,12.000000,1.284289,19.769230,14.709678,61.900002,6.300000,8.080000,8.700000,7.300000,6.660000,6.857143,6.710000,7.00000,21.700001,0.000000
30%,2.000000,2.532599,1482.000000,0.000000,0.790532,0.174986,0.016810,0.013238,0.012684,0.000778,0.493053,0.479837,364.500000,274.000000,14.000000,1.314428,21.358655,15.666667,65.730003,6.900000,8.570000,9.300000,8.100000,7.245000,7.666667,6.970000,7.00000,23.400000,0.000000
40%,3.000000,2.775798,1700.000000,0.000835,0.793531,0.177907,0.018519,0.014957,0.013810,0.001460,0.514706,0.492891,420.000000,310.000000,16.000000,1.339181,22.853659,16.727272,68.599998,7.400000,9.100000,9.700000,8.900000,7.770000,8.333333,7.230000,8.00000,24.799999,0.000000


In [13]:
features.sort()
with open(f"output/features_{version}.json", "w") as f:
    json.dump({"feature_names": features}, f)

In [14]:
df.to_parquet(f"output/features_{version}.parquet", index=False)
assert df.notna().all(axis=None)

In [15]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 0:01:00.290712
